In [1]:
from random import random, seed
import numpy as np
from random import randrange
from csv import reader

In [2]:
# creation of the architecture.
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [3]:
# activation = X1*W1 + X2*W2 + ... + Xn*Wn + bias.
def activate(weights, inputs):
    
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [4]:
# choose the activation function, sigmoid is the function by default.
def transfer(activation, function):
    
    if function == 'tanh':
        return (np.exp(activation) - np.exp(-activation)) / (np.exp(activation) + np.exp(-activation))
    elif function == 'relu':
        if activation > 0:
            return activation
        else:
            return 0
    elif function == 'arctan':
        return np.arctan(activation)
    elif function == 'softplus':
        return np.log(1 + np.exp(activation))
    else:
        return 1 / (1 + np.exp(-activation))

In [5]:
# compute of the outputs.
def forward_propagate(network, row, function):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['activation'] = activation
            neuron['output'] = transfer(activation, function)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [6]:
# there is the derivative of the functions use for the activation. See: https://en.wikipedia.org/wiki/Activation_function
def transfer_derivative(output, function, activation):
    
    if function == 'tanh':
        return 1 - output**2
    elif function == 'relu':
        if activation > 0:
            return 1
        else:
            return 0
    elif function == 'arctan':
        return 1/(activation**2 + 1)
    elif function == 'softplus':
        return 1/(1 + np.exp(-activation))
    else:
        return output * (1 - output)

In [7]:
# compute the errors in function of the expected outputs.
def backward_propagate_error(network, expected, function):
    
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)): 
                error = 0
                for neuron in network[i + 1]: 
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], function,neuron['activation'])

In [8]:
# change the value of the weights after the back-propagation.
def update_weights(network, row, l_rate):
    
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [9]:
# application of the algorithm for each value of a dataset. It's repeat several time.
def train_network(network, train, l_rate, n_epoch, n_outputs,function):
    
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row, function)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected, function)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        

In [10]:
# it permits us to compute some data to get a result in function of a network.
def predict(network, row, function):
    
    outputs = forward_propagate(network, row, function)
    return outputs.index(max(outputs))

In [11]:
# loading a csv file.
def load_csv(filename):
    
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [12]:
# convert string column to float.
def str_column_to_float(dataset, column):
    
    for row in dataset:
        row[column] = float(row[column].strip())

In [13]:
# convert string column to integer.
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [14]:
# find the min and max values for each column.
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

In [15]:
# rescale dataset columns to the range 0-1.
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [16]:
# split a dataset into k folds.
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [17]:
# calculate accuracy percentage.
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [18]:
# evaluate an algorithm using a cross validation split.
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [19]:
# backpropagation Algorithm With Stochastic Gradient Descent.
def back_propagation(train, test, l_rate, n_epoch, n_hidden, function):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs, function)
    predictions = list()
    for row in test:
        prediction = predict(network, row, function)
        predictions.append(prediction)
    return(predictions)

In [20]:
# set a seed for the random.
seed(60198)

In [21]:
filename = 'webData.csv'
dataset = load_csv(filename)

In [22]:
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)

In [23]:
str_column_to_int(dataset, len(dataset[0])-1)

{'3': 0, '2': 1, '1': 2}

In [24]:
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

In [25]:
# allows us to see 
def trainings(n_folds, l_rate, n_epoch, n_hidden, function):
    
    scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden, function)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
    print('Activation function: %s' % function)

In [26]:
trainings(5, 0.3, 500, 3, 'relu')

>epoch=0, lrate=0.300, error=200.218
>epoch=1, lrate=0.300, error=168.000
>epoch=2, lrate=0.300, error=168.000
>epoch=3, lrate=0.300, error=168.000
>epoch=4, lrate=0.300, error=168.000
>epoch=5, lrate=0.300, error=168.000
>epoch=6, lrate=0.300, error=168.000
>epoch=7, lrate=0.300, error=168.000
>epoch=8, lrate=0.300, error=168.000
>epoch=9, lrate=0.300, error=168.000
>epoch=10, lrate=0.300, error=168.000
>epoch=11, lrate=0.300, error=168.000
>epoch=12, lrate=0.300, error=168.000
>epoch=13, lrate=0.300, error=168.000
>epoch=14, lrate=0.300, error=168.000
>epoch=15, lrate=0.300, error=168.000
>epoch=16, lrate=0.300, error=168.000
>epoch=17, lrate=0.300, error=168.000
>epoch=18, lrate=0.300, error=168.000
>epoch=19, lrate=0.300, error=168.000
>epoch=20, lrate=0.300, error=168.000
>epoch=21, lrate=0.300, error=168.000
>epoch=22, lrate=0.300, error=168.000
>epoch=23, lrate=0.300, error=168.000
>epoch=24, lrate=0.300, error=168.000
>epoch=25, lrate=0.300, error=168.000
>epoch=26, lrate=0.300

>epoch=219, lrate=0.300, error=168.000
>epoch=220, lrate=0.300, error=168.000
>epoch=221, lrate=0.300, error=168.000
>epoch=222, lrate=0.300, error=168.000
>epoch=223, lrate=0.300, error=168.000
>epoch=224, lrate=0.300, error=168.000
>epoch=225, lrate=0.300, error=168.000
>epoch=226, lrate=0.300, error=168.000
>epoch=227, lrate=0.300, error=168.000
>epoch=228, lrate=0.300, error=168.000
>epoch=229, lrate=0.300, error=168.000
>epoch=230, lrate=0.300, error=168.000
>epoch=231, lrate=0.300, error=168.000
>epoch=232, lrate=0.300, error=168.000
>epoch=233, lrate=0.300, error=168.000
>epoch=234, lrate=0.300, error=168.000
>epoch=235, lrate=0.300, error=168.000
>epoch=236, lrate=0.300, error=168.000
>epoch=237, lrate=0.300, error=168.000
>epoch=238, lrate=0.300, error=168.000
>epoch=239, lrate=0.300, error=168.000
>epoch=240, lrate=0.300, error=168.000
>epoch=241, lrate=0.300, error=168.000
>epoch=242, lrate=0.300, error=168.000
>epoch=243, lrate=0.300, error=168.000
>epoch=244, lrate=0.300, 

>epoch=433, lrate=0.300, error=168.000
>epoch=434, lrate=0.300, error=168.000
>epoch=435, lrate=0.300, error=168.000
>epoch=436, lrate=0.300, error=168.000
>epoch=437, lrate=0.300, error=168.000
>epoch=438, lrate=0.300, error=168.000
>epoch=439, lrate=0.300, error=168.000
>epoch=440, lrate=0.300, error=168.000
>epoch=441, lrate=0.300, error=168.000
>epoch=442, lrate=0.300, error=168.000
>epoch=443, lrate=0.300, error=168.000
>epoch=444, lrate=0.300, error=168.000
>epoch=445, lrate=0.300, error=168.000
>epoch=446, lrate=0.300, error=168.000
>epoch=447, lrate=0.300, error=168.000
>epoch=448, lrate=0.300, error=168.000
>epoch=449, lrate=0.300, error=168.000
>epoch=450, lrate=0.300, error=168.000
>epoch=451, lrate=0.300, error=168.000
>epoch=452, lrate=0.300, error=168.000
>epoch=453, lrate=0.300, error=168.000
>epoch=454, lrate=0.300, error=168.000
>epoch=455, lrate=0.300, error=168.000
>epoch=456, lrate=0.300, error=168.000
>epoch=457, lrate=0.300, error=168.000
>epoch=458, lrate=0.300, 

>epoch=146, lrate=0.300, error=168.000
>epoch=147, lrate=0.300, error=168.000
>epoch=148, lrate=0.300, error=168.000
>epoch=149, lrate=0.300, error=168.000
>epoch=150, lrate=0.300, error=168.000
>epoch=151, lrate=0.300, error=168.000
>epoch=152, lrate=0.300, error=168.000
>epoch=153, lrate=0.300, error=168.000
>epoch=154, lrate=0.300, error=168.000
>epoch=155, lrate=0.300, error=168.000
>epoch=156, lrate=0.300, error=168.000
>epoch=157, lrate=0.300, error=168.000
>epoch=158, lrate=0.300, error=168.000
>epoch=159, lrate=0.300, error=168.000
>epoch=160, lrate=0.300, error=168.000
>epoch=161, lrate=0.300, error=168.000
>epoch=162, lrate=0.300, error=168.000
>epoch=163, lrate=0.300, error=168.000
>epoch=164, lrate=0.300, error=168.000
>epoch=165, lrate=0.300, error=168.000
>epoch=166, lrate=0.300, error=168.000
>epoch=167, lrate=0.300, error=168.000
>epoch=168, lrate=0.300, error=168.000
>epoch=169, lrate=0.300, error=168.000
>epoch=170, lrate=0.300, error=168.000
>epoch=171, lrate=0.300, 

>epoch=368, lrate=0.300, error=168.000
>epoch=369, lrate=0.300, error=168.000
>epoch=370, lrate=0.300, error=168.000
>epoch=371, lrate=0.300, error=168.000
>epoch=372, lrate=0.300, error=168.000
>epoch=373, lrate=0.300, error=168.000
>epoch=374, lrate=0.300, error=168.000
>epoch=375, lrate=0.300, error=168.000
>epoch=376, lrate=0.300, error=168.000
>epoch=377, lrate=0.300, error=168.000
>epoch=378, lrate=0.300, error=168.000
>epoch=379, lrate=0.300, error=168.000
>epoch=380, lrate=0.300, error=168.000
>epoch=381, lrate=0.300, error=168.000
>epoch=382, lrate=0.300, error=168.000
>epoch=383, lrate=0.300, error=168.000
>epoch=384, lrate=0.300, error=168.000
>epoch=385, lrate=0.300, error=168.000
>epoch=386, lrate=0.300, error=168.000
>epoch=387, lrate=0.300, error=168.000
>epoch=388, lrate=0.300, error=168.000
>epoch=389, lrate=0.300, error=168.000
>epoch=390, lrate=0.300, error=168.000
>epoch=391, lrate=0.300, error=168.000
>epoch=392, lrate=0.300, error=168.000
>epoch=393, lrate=0.300, 

...

>epoch=36, lrate=0.300, error=155.443
>epoch=37, lrate=0.300, error=155.443
>epoch=38, lrate=0.300, error=155.443
>epoch=39, lrate=0.300, error=155.443
>epoch=40, lrate=0.300, error=155.443
>epoch=41, lrate=0.300, error=155.443
>epoch=42, lrate=0.300, error=155.443
>epoch=43, lrate=0.300, error=155.443
>epoch=44, lrate=0.300, error=155.443
>epoch=45, lrate=0.300, error=155.443
>epoch=46, lrate=0.300, error=155.443
>epoch=47, lrate=0.300, error=155.443
>epoch=48, lrate=0.300, error=155.443
>epoch=49, lrate=0.300, error=155.443
>epoch=50, lrate=0.300, error=155.443
>epoch=51, lrate=0.300, error=155.443
>epoch=52, lrate=0.300, error=155.443
>epoch=53, lrate=0.300, error=155.443
>epoch=54, lrate=0.300, error=155.443
>epoch=55, lrate=0.300, error=155.443
>epoch=56, lrate=0.300, error=155.443
>epoch=57, lrate=0.300, error=155.443
>epoch=58, lrate=0.300, error=155.443
>epoch=59, lrate=0.300, error=155.443
>epoch=60, lrate=0.300, error=155.443
>epoch=61, lrate=0.300, error=155.443
>epoch=62, l

>epoch=250, lrate=0.300, error=155.443
>epoch=251, lrate=0.300, error=155.443
>epoch=252, lrate=0.300, error=155.443
>epoch=253, lrate=0.300, error=155.443
>epoch=254, lrate=0.300, error=155.443
>epoch=255, lrate=0.300, error=155.443
>epoch=256, lrate=0.300, error=155.443
>epoch=257, lrate=0.300, error=155.443
>epoch=258, lrate=0.300, error=155.443
>epoch=259, lrate=0.300, error=155.443
>epoch=260, lrate=0.300, error=155.443
>epoch=261, lrate=0.300, error=155.443
>epoch=262, lrate=0.300, error=155.443
>epoch=263, lrate=0.300, error=155.443
>epoch=264, lrate=0.300, error=155.443
>epoch=265, lrate=0.300, error=155.443
>epoch=266, lrate=0.300, error=155.443
>epoch=267, lrate=0.300, error=155.443
>epoch=268, lrate=0.300, error=155.443
>epoch=269, lrate=0.300, error=155.443
>epoch=270, lrate=0.300, error=155.443
>epoch=271, lrate=0.300, error=155.443
>epoch=272, lrate=0.300, error=155.443
>epoch=273, lrate=0.300, error=155.443
>epoch=274, lrate=0.300, error=155.443
>epoch=275, lrate=0.300, 

>epoch=471, lrate=0.300, error=155.443
>epoch=472, lrate=0.300, error=155.443
>epoch=473, lrate=0.300, error=155.443
>epoch=474, lrate=0.300, error=155.443
>epoch=475, lrate=0.300, error=155.443
>epoch=476, lrate=0.300, error=155.443
>epoch=477, lrate=0.300, error=155.443
>epoch=478, lrate=0.300, error=155.443
>epoch=479, lrate=0.300, error=155.443
>epoch=480, lrate=0.300, error=155.443
>epoch=481, lrate=0.300, error=155.443
>epoch=482, lrate=0.300, error=155.443
>epoch=483, lrate=0.300, error=155.443
>epoch=484, lrate=0.300, error=155.443
>epoch=485, lrate=0.300, error=155.443
>epoch=486, lrate=0.300, error=155.443
>epoch=487, lrate=0.300, error=155.443
>epoch=488, lrate=0.300, error=155.443
>epoch=489, lrate=0.300, error=155.443
>epoch=490, lrate=0.300, error=155.443
>epoch=491, lrate=0.300, error=155.443
>epoch=492, lrate=0.300, error=155.443
>epoch=493, lrate=0.300, error=155.443
>epoch=494, lrate=0.300, error=155.443
>epoch=495, lrate=0.300, error=155.443
>epoch=496, lrate=0.300, 

>epoch=185, lrate=0.300, error=168.000
>epoch=186, lrate=0.300, error=168.000
>epoch=187, lrate=0.300, error=168.000
>epoch=188, lrate=0.300, error=168.000
>epoch=189, lrate=0.300, error=168.000
>epoch=190, lrate=0.300, error=168.000
>epoch=191, lrate=0.300, error=168.000
>epoch=192, lrate=0.300, error=168.000
>epoch=193, lrate=0.300, error=168.000
>epoch=194, lrate=0.300, error=168.000
>epoch=195, lrate=0.300, error=168.000
>epoch=196, lrate=0.300, error=168.000
>epoch=197, lrate=0.300, error=168.000
>epoch=198, lrate=0.300, error=168.000
>epoch=199, lrate=0.300, error=168.000
>epoch=200, lrate=0.300, error=168.000
>epoch=201, lrate=0.300, error=168.000
>epoch=202, lrate=0.300, error=168.000
>epoch=203, lrate=0.300, error=168.000
>epoch=204, lrate=0.300, error=168.000
>epoch=205, lrate=0.300, error=168.000
>epoch=206, lrate=0.300, error=168.000
>epoch=207, lrate=0.300, error=168.000
>epoch=208, lrate=0.300, error=168.000
>epoch=209, lrate=0.300, error=168.000
>epoch=210, lrate=0.300, 

>epoch=395, lrate=0.300, error=168.000
>epoch=396, lrate=0.300, error=168.000
>epoch=397, lrate=0.300, error=168.000
>epoch=398, lrate=0.300, error=168.000
>epoch=399, lrate=0.300, error=168.000
>epoch=400, lrate=0.300, error=168.000
>epoch=401, lrate=0.300, error=168.000
>epoch=402, lrate=0.300, error=168.000
>epoch=403, lrate=0.300, error=168.000
>epoch=404, lrate=0.300, error=168.000
>epoch=405, lrate=0.300, error=168.000
>epoch=406, lrate=0.300, error=168.000
>epoch=407, lrate=0.300, error=168.000
>epoch=408, lrate=0.300, error=168.000
>epoch=409, lrate=0.300, error=168.000
>epoch=410, lrate=0.300, error=168.000
>epoch=411, lrate=0.300, error=168.000
>epoch=412, lrate=0.300, error=168.000
>epoch=413, lrate=0.300, error=168.000
>epoch=414, lrate=0.300, error=168.000
>epoch=415, lrate=0.300, error=168.000
>epoch=416, lrate=0.300, error=168.000
>epoch=417, lrate=0.300, error=168.000
>epoch=418, lrate=0.300, error=168.000
>epoch=419, lrate=0.300, error=168.000
>epoch=420, lrate=0.300, 